### Illustrative Example Script

The Illustrative Example data is from six fixed cameras on top a 43-m tower (known as the Argus Tower) at the Army Corps of Engineers Field Research Facility in Duck, NC. 

Each full oblique video is 1 minute long and captured at 2 frames per second. Each video was recorded simultaneously. Six separate grayscale .avi files, corresponding to each camera collection, are included. 

Single frames are provided to demonstrate calling mergeRectify() directly. These 6 oblique frames were taken from the .avi files provided, and are included in color and grayscale.

Finally, a set of 5 frames for each camera is included to demonstrate calling mergeRectify() in a loop to rectify images at subsequent timestamps.

Extrinsic and intrinsic values for each camera are provided in both direct linear transform coefficient format as well as in CIRN convention.

Let's start with our imports. 

Ensure that each package is installed prior to running your script, in addition to the package dependencies described in section 2 of the CoastalImageLib User Manual. 
Suggested install is via 'pip install ___'

In [2]:
# CoastalImageLib imports
import corefunctions as cf
import supportfunctions as sf

# External imports
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import imageio

Now, we can initialize the strings that contain our file information.

In [7]:
# Image, camera data, and video locations
folder = 'C:/Users/mccan/Desktop/MiscWAM/304_Oct.31/'

# These are the 'tags' for each camera. They are used
# to distinguish between camera files, as well as
# intrinsic and extrinsic values contained in the .yaml file, 
# loaded in the next code block.
cams = ['c1','c2','c3','c4','c5','c6']

In [14]:
# This is our list of strings containing the file name and
# locations for each FRAME (.jpg file) file we want to rectify

# Grayscale frames
image_list_gray = [folder + i + '_frame.jpg' for i in cams]

# RGB frames
image_list_rgb = [folder + i + '_frame_rgb.jpg' for i in cams]

In [15]:
# This is our list of strings containing the file name and
# locations for each VIDEO (.avi file) we want to rectify
video_list = [folder + '1604152800.Sat.Oct.31_14_00_00.GMT.2020.' + i + '.avi' for i in cams]

Next, let's load our intrinsic and extrinsic values, as well as relevant camera metadata, into our "camera" objects. We do this by initializing an array of CameraData objects with our calibration information. 

In [10]:
# Location of our calibration data, stored in a .yaml file 
calibration_loc = folder + 'cameraData.yml'

# Call a support function to format .yaml files into CIRN convention
m, ex = sf.loadYamlDLT(calibration_loc,cams) 

Tags:
coords = 'local', because our calibration data is already in local (FRF) coordinates
origin = 'None', because we don't need to specify a local origin since we are already in local coordinates
mType = 'DLT', because our cameras were calibrated using the Direct Linear Transform method. Most cameras will not be calibrated this way. For more information on each format of intrinsic values, see the CoastalImageLib user manual.

In [11]:
cameras = np.empty(len(cams),dtype=object)
for i in range(len(cams)):
    cameras[i] = cf.CameraData(m[i], ex[i], coords = 'local', origin= 'None', mType = 'DLT',nc=3)

The next step is to initialize the rectification grid. This is the target grid on which the nadir image will appear. The grid is created using the XYZGrid object.

In [21]:
# Grid boundaries
xMin = 0
xMax = 500
yMin = -500
yMax = 1200
dy = 1
dx = 1
z = 0

grid = cf.XYZGrid([xMin,xMax], [yMin,yMax], dx, dy, z)

## Rectifying Oblique Frames

Let's first try rectifying single frames from each camera and merging the images in both grayscale in color. In python, the main difference between grayscale and color images is the number of channels, or the depth of the image matrix. CoastalImageLib can accomodate both grayscale (1 channel) and color/ RGB (3 channels).  

First, in grayscale:

In [22]:
rect_frame_gray = cf.mergeRectify(image_list_gray, cameras, grid)

In [23]:
# Display our rectified frame
# Click 'x' on the pop- up window to leave display
cv.imwrite('C:/Users/mccan/Desktop/drifterbackground.jpg',rect_frame_gray.astype(np.uint8))
cv.imshow('Rectified Frame, Gray',rect_frame_gray.astype(np.uint8))
cv.waitKey(0)

-1

Next, in color:

In [ ]:
rect_frame_rgb = cf.mergeRectify(image_list_rgb, cameras, grid)

In [ ]:
# Display our rectified frame
# Click 'x' on the pop- up window to leave display
cv.imshow('Rectified Frame, Color',rect_frame_rgb.astype(np.uint8))
cv.waitKey(0)

## Rectifying Multiple Oblique Frames

What if we want to rectify a sequence of oblique frames, for example, videos that have been converted to single frames.

In this case, we need to set up a loop to go through each *timestamp* of the frames, and merge all the cameras at that *timestamp*.

Ultimately, you will have to write your own for loop to accomodate your specific application.

In [ ]:
# Here, we will only work with 5 frames per camera, each sampled one second apart. 

start_time = 1604152800 # Timestamp of the first frame, in UTC time
end_time = start_time + 5 # Timestamp of the last frame, in UTC time

# The timestamp of each frame is contained in the file name
file_changes = np.arange(start_time, end_time)

for f in file_changes:
    curr_image_list = [folder + str(f) + '_' + i + '_frame_gray.jpg' for i in cams]
    curr_rect_frame = cf.mergeRectify(curr_image_list, cameras, grid)
    
    # Display the current frame
    # Click 'x' on the pop- up window to move on to next frame
    disp_str = 'Merged and Rectified Frame ' + str(f)
    cv.imshow(disp_str, curr_rect_frame)
    cv.waitKey(0)
    

## Rectifying Oblique Videos

Finally, let's rectify our videos!

numFrames is the number of frames we would like to rectify. Here, our video is sampled at 2 Hz, and we would like 10 seconds of data. Therefore, numFrames = 2*10

In [ ]:
numFrames = 20 # 2 fps for 10 seconds

# Call rectVideos !
rect = cf.rectVideos(video_list, cameras, grid, numFrames)

Now, you have an object containing your rectified frames. However, your video is saved to your drive in your working directory as the input file name + 'video_capture.rect.avi'.

## Creating a Pixel Timestack

Let's create subsampled pixel timestacks. These timestacks are for use in algorithms such as bathymetric inversion, surface current estimation, or run-up calculations. Pixel timestacks show variations in pixel intensity over time.

In [ ]:
# Here, we will create one pixel stack at a single timestamp and display the location on oblique images

# Ultimately, you will need to write your own for loop to accomodate your specific application

# Click 'x' to exit image displays

# Alongshore Transect
x = 300
yMin = 900
yMax = 1200
dy = 5
alongshore_trans = cf.XYZGrid([x], [yMin,yMax], dx, dy, z)
rect = cf.pixelStack(im_list, alongshore_trans, cameras, disp_flag=1)

In [ ]:
# Cross- shore Transect
xMin = 100
xMax = 0
y = 1200
dx = 5
alongshore_trans = cf.XYZGrid([xMin, xMax], [y], dx, dy, z)
rect = cf.pixelStack(im_list, alongshore_trans, cameras, disp_flag=1)


In [ ]:
# Pixel Grid
xMin = 400
xMax = 150
yMin = 500
yMax = 1200
dx = 5
dy = 5
alongshore_trans = cf.XYZGrid([xMin, xMax], [y], dx, dy, z)
rect = cf.pixelStack(im_list, alongshore_trans, cameras, disp_flag=1)

## Calculating Image Statistics

Lastly, let's calculate sample statistical image products. These products include Darkest, Brightest, Timex, and Variance from an array of images. For detailed information, see the accompanying SoftwareX publication, or the CoastalImageLib User Manual.

In [ ]:
# Let's find the image statistics from the rectified array from our videos
# Save flag indicates that the resulting image products will be saved 
# in the current working directory

# Click 'x' to exit image display
cf.imageStats(rect, save_flag = 1)